In [1]:
from datetime import date, datetime, timezone
import os
import warnings

import git
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import esmlab
import esmlab_regrid
import pop_tools

from gen_remap_weights_1d import gen_remap_weights_1d
import config
import util

In [2]:
# script identifier to go in generated file

repo = 'github.com/marbl-ecosys/marbl-forcing'
sha = git.Repo(search_parent_directories=True).head.object.hexsha
script_fname = 'initial_conditions/gen_cesm2_omip_IC.ipynb'
id_string = f'{repo}/tree/{sha}/{script_fname}'

In [3]:
# input specifications

tracernames = [ 'TEMP', 'SALT',
    'NO3', 'PO4', 'SiO3', 'O2', 'DIC', 'ALK', 
    'spC', 'spChl', 'spFe', 'spP', 'spCaCO3', 
    'diatC', 'diatChl', 'diatFe', 'diatP', 'diatSi', 
    'diazC', 'diazChl', 'diazFe', 'diazP', 
    'zooC', 
    'DOC', 'DOCr', 'DON', 'DONr', 'DOP', 'DOPr', 'NH4', 
    'Fe', 'Lig'
]

# IC specified by OMIP (macronutrients, O2, DIC, ALK)
dirin_omip = '/glade/p/cesm/bgcwg/OMIPinit'
omip_metadata = {
    'TEMP':{'fname':'woa13_temperature_OMIPinit.nc', 'varname':'temp'},
    'SALT':{'fname':'woa13_salinity_OMIPinit.nc', 'varname':'salt'},
    'NO3':{'fname':'woa13_nitrate_OMIPinit.nc', 'varname':'no3'},
    'PO4':{'fname':'woa13_phosphate_OMIPinit.nc', 'varname':'po4'},
    'SiO3':{'fname':'woa13_silicate_OMIPinit.nc', 'varname':'si'},
    'O2':{'fname':'woa13_oxygen_OMIPinit.nc', 'varname':'o2'},
    'DIC':{'fname':'glodapv2_Ct_preind_OMIPinit.nc', 'varname':'Ct_preind'},
    'ALK':{'fname':'glodapv2_At_OMIPinit.nc', 'varname':'At'}
}

# Add grid name to metadata.
# This is fragile. It would be better to determine this automatically.
# Alternatively, the script would be more self contained if the grid were constructed on the fly.
omip_metadata['TEMP']['grid'] = 'latlon_1x1_180W'
omip_metadata['SALT']['grid'] = 'latlon_1x1_180W'
omip_metadata['NO3']['grid'] = 'latlon_1x1_180W'
omip_metadata['PO4']['grid'] = 'latlon_1x1_180W'
omip_metadata['SiO3']['grid'] = 'latlon_1x1_180W'
omip_metadata['O2']['grid'] = 'latlon_1x1_180W'
omip_metadata['DIC']['grid'] = 'latlon_1x1_20E'
omip_metadata['ALK']['grid'] = 'latlon_1x1_20E'

# get other tracers (living pools, DOM, NH4, Fe, Lig) from refcase
refcase = 'b.e21.B1850.f09_g17.CMIP6-piControl.001'
refdate = '0501-01-01'
dirin_refcase = f'{config.inputdata}/cesm2_init/{refcase}/{refdate}'
fname_refcase = f'{dirin_refcase}/{refcase}.pop.r.{refdate}-00000.nc'
print(fname_refcase)

/glade/p/cesmdata/cseg/inputdata/cesm2_init/b.e21.B1850.f09_g17.CMIP6-piControl.001/0501-01-01/b.e21.B1850.f09_g17.CMIP6-piControl.001.pop.r.0501-01-01-00000.nc


In [4]:
# details on file being generated

dst_grid = 'POP_gx1v7'
ltripole = False
datestamp = date.today().strftime("%y%m%d")
fname_out = f'{config.dirout}/ecosys_jan_IC_omip_{dst_grid}_c{datestamp}.nc'

In [5]:
# start ds_out as Dataset of grid variables

ds_out = pop_tools.get_grid(grid_name=dst_grid)

# add z_t_bounds
z_t_edges = np.concatenate(([0.0], np.cumsum(ds_out.dz)))
z_t_bounds_vals = np.stack((z_t_edges[:-1], z_t_edges[1:]), axis=1)
ds_out['z_t_bounds'] = xr.DataArray(z_t_bounds_vals, dims=('z_t', 'd2'),
                                   coords={'z_t':ds_out.z_t})
ds_out['z_t'].attrs['bounds'] = 'z_t_bounds'

In [6]:
# generate output grid 3D mask

nk = len(ds_out.z_t)
nj, ni = ds_out.KMT.shape

# make 3D array of 0:km
zero_to_km = xr.DataArray(np.arange(0, nk), dims=('z_t'))
ONES_3d = xr.DataArray(np.ones((nk, nj, ni)), dims=('z_t', 'nlat', 'nlon'))
ZERO_TO_KM = (zero_to_km * ONES_3d)

# mask out cells where k is below KMT
MASK = ZERO_TO_KM.where(ZERO_TO_KM < ds_out.KMT)
MASK = xr.where(MASK.notnull(), True, False)

MASK['z_t'] = ds_out.z_t

In [7]:
USER = os.environ['USER']
gridfile_directory = esmlab.config.set({'regrid.gridfile-directory': f'/glade/work/{USER}/esmlab-regrid'})

for tracername in omip_metadata:
    # decode_times=False because omip files have unsupported units
    #     "months since 0000-01-01 00:00:00"
    ds_omip = xr.open_dataset(dirin_omip+'/'+omip_metadata[tracername]['fname'], decode_times=False)
    varname_omip = omip_metadata[tracername]['varname']

    print(f'regridding {tracername}')

    # perform lateral remapping
    regrid_operator = esmlab_regrid.regridder(
        name_grid_src=omip_metadata[tracername]['grid'], 
        name_grid_dst=dst_grid, method='conservative', overwrite_existing=False)
    tracer_omip = ds_omip[varname_omip]
    if 'time' in tracer_omip.dims:
        tracer_omip = tracer_omip.isel(time=0, drop=True)
    tracer_lateral_regrid = regrid_operator(tracer_omip, renormalize=True)

    # generate vertical remapping weights
    depth_name_in = tracer_omip.dims[0]
    depth_bounds_name = ds_omip[depth_name_in].attrs['bounds']
    depth_remap_weights = gen_remap_weights_1d(
        ds_omip[depth_bounds_name], ds_out['z_t_bounds'],
        ds_omip[depth_name_in].attrs['units'], ds_out.dz.attrs['units'])

    # perform vertical remapping
    nk_in = tracer_lateral_regrid.shape[0]
    tracer_lateral_regrid_flat = tracer_lateral_regrid.values.reshape((nk_in, nj*ni))
    tracer_full_regrid_flat = depth_remap_weights.dot(tracer_lateral_regrid_flat)
    tracer_full_regrid_values = tracer_full_regrid_flat.reshape((nk, nj, ni))
    tracer_full_regrid = xr.DataArray(tracer_full_regrid_values, dims=('z_t', 'nlat', 'nlon'),
                                      coords={'z_t':ds_out.z_t}, attrs=tracer_omip.attrs)
    tracer_full_regrid = tracer_full_regrid.where(MASK)
    tracer_full_regrid.encoding['_FillValue'] = tracer_omip.encoding['_FillValue']
    
    # perform lateral fill
    tracer_full_regrid = pop_tools.lateral_fill(tracer_full_regrid, MASK, ltripole=ltripole)

    # perform vertical fill
    unfilled = tracer_full_regrid.isnull() & (MASK == 1.)                    
    if unfilled.any():
        for k in range(1, nk):
            tracer_full_regrid[k, :, :] = xr.where(unfilled[k, :, :],
                                                   tracer_full_regrid[k-1, :, :],
                                                   tracer_full_regrid[k, :, :])
        if (tracer_full_regrid.isnull() & (MASK == 1.)).any():
            warnings.warn(f'{tracername} still has missing values')

    ds_out[tracername] = tracer_full_regrid
ds_out

regridding TEMP
regridding SALT
regridding NO3
regridding PO4
regridding SiO3
regridding O2
regridding DIC


/glade/work/klindsay/miniconda3/envs/analysis_dev/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: DIC still has missing values


regridding ALK


/glade/work/klindsay/miniconda3/envs/analysis_dev/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: ALK still has missing values


<xarray.Dataset>
Dimensions:      (d2: 2, nlat: 384, nlon: 320, z_t: 60, z_w: 60, z_w_bot: 60)
Coordinates:
  * z_t          (z_t) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
  * z_w          (z_w) float64 0.0 1e+03 2e+03 3e+03 ... 4.75e+05 5e+05 5.25e+05
  * z_w_bot      (z_w_bot) float64 1e+03 2e+03 3e+03 ... 5e+05 5.25e+05 5.5e+05
Dimensions without coordinates: d2, nlat, nlon
Data variables:
    TLAT         (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG        (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
    ULAT         (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG        (nlat, nlon) float64 321.1 322.3 323.4 ... 319.2 319.6 320.0
    DXT          (nlat, nlon) float64 2.339e+06 2.339e+06 ... 1.473e+06
    DYT          (nlat, nlon) float64 5.94e+06 5.94e+06 ... 5.046e+06 5.046e+06
    TAREA        (nlat, nlon) float64 1.39e+13 1.39e+13 ... 7.431e+12 7.432e+12
    KMT          (nlat, nlon) int32 0 0 0 

In [8]:
# special treatment of DIC and ALK in marginal seas
SALT = ds_out.SALT
DIC = ds_out.DIC
ALK = ds_out.ALK
rho_sw = 1.026

# Black Sea
# fit by eye from Fig 6 of Goyet, Bradshaw, Brewer
# The carbonate system in the Black Sea
# Deep-Sea Research, part A, Vol 38, Suppl 2, 1991, pp. S1049-S1068
MASK_fit = MASK & (ds_out.REGION_MASK == -13) & (SALT <= 21.0)
ALK = xr.where(MASK_fit, 3250.0, ALK)
DIC = xr.where(MASK_fit, 3250.0, DIC)
MASK_fit = MASK & (ds_out.REGION_MASK == -13) & (SALT <= 19.5)
DIC = xr.where(MASK_fit, 3000.0 + (SALT - 18) / (19.5 - 18) * (3250 - 3000), DIC)
MASK_fit = MASK & (ds_out.REGION_MASK == -13) & (SALT > 21.0)
ALK = xr.where(MASK_fit, 3300.0 + (SALT - 21) / (22.3 - 21) * (4500 - 3300), ALK)
DIC = xr.where(MASK_fit, (ALK + 1000) / 1.32, DIC)

# Baltic Sea
# Hjalmarsson et al, Distribution, long-term development and mass
# balance calculation of total alkalinity in the Baltic Sea,
# Continental Shelf Research, Vol 28, 2008, pp. 593-601
# linear fits from Table 2
# one fit for psu < 7.3 for Gulf of Bothnia and Bothnian Sea
# another fit for Baltic Proper
# ignore values from Gulf of Finland and Gulf of Riga
# paper reports in umol/L, convert to umol/kg
MASK_fit = MASK & (ds_out.REGION_MASK == -12) & (SALT <= 7.3)
ALK = xr.where(MASK_fit, (119.0 + 196.0 * SALT) / rho_sw, ALK)
DIC = xr.where(MASK_fit, ALK / 1.01, DIC)
MASK_fit = MASK & (ds_out.REGION_MASK == -12) & (SALT > 7.3)
ALK = xr.where(MASK_fit, (1237.0 + 43.0 * SALT) / rho_sw, ALK)
DIC = xr.where(MASK_fit, ALK / 1.01, DIC)

ds_out['DIC'].values = DIC.values
ds_out['ALK'].values = ALK.values

In [9]:
# copy from refcase restart file to ds_out tracers not yet in ds_out

fillvalue = -1.0e34 # value to replace land values with

ds_refcase = xr.open_dataset(fname_refcase).rename({'k':'z_t', 'j':'nlat', 'i':'nlon'})
ds_refcase['z_t'] = ds_out['z_t']
for tracername in tracernames:
    if tracername not in ds_out.data_vars:
        print(f'copying {tracername} from {refcase} restart file')
        tracer = ds_refcase[tracername+'_CUR']
        tracer = tracer.where(MASK)
        # Set floor values that are 0.0 to value 1 layer shallower.
        # This is to fill in values where KMT was popped up by overflow parameterization.
        # This is needed to enable using ds_out in runs where overflows are disabled.
        for k in range(1, nk):
            tracer[k, :, :] = xr.where((tracer[k, :, :] == 0.0) & (k+1 == ds_out.KMT),
                                       tracer[k-1, :, :], tracer[k, :, :])
        tracer.attrs['coordinates'] = 'TLONG TLAT z_t'
        tracer.encoding['_FillValue'] = fillvalue
        ds_out[tracername] = tracer

copying spC from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spChl from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spFe from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spP from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying spCaCO3 from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatC from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatChl from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatFe from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatP from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diatSi from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diazC from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diazChl from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diazFe from b.e21.B1850.f09_g17.CMIP6-piControl.001 restart file
copying diazP from b.e21.B1850.f09_g17.CMIP6-piControl.001 r

In [10]:
# set derived quantities

ds_out['DIC_ALT_CO2'] = ds_out['DIC']
ds_out['ALK_ALT_CO2'] = ds_out['ALK']
DOCtot = ds_out['DOC']
DOCtot.values = ds_out['DOC'].values + ds_out['DOCr'].values
DOCtot.attrs['long_name'] = 'DOC + DOCr'
ds_out['DOCtot'] = DOCtot

In [11]:
# write generated file

datestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d")
ds_out.attrs['history'] = f'created by {id_string} on {datestamp}'
omip_file_list = [dirin_omip+'/'+omip_metadata[varname]['fname'] for varname in omip_metadata]
ds_out.attrs['input_file_list'] = ' '.join(omip_file_list + [fname_refcase])
util.ds_clean(ds_out).to_netcdf(fname_out)